In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [17]:
# Load datasets
brent = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\brent_inr.csv")
wti = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\wti_inr.csv")

nifty50 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty50.csv")
nifty100 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty100.csv")
nifty200 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty200.csv")
nifty500 = pd.read_csv(r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion\Nifty500.csv")

In [18]:
# Preview top rows and columns
datasets = {
    "Brent": brent,
    "WTI": wti,
    "Nifty50": nifty50,
    "Nifty100": nifty100,
    "Nifty200": nifty200,
    "Nifty500": nifty500
}

In [19]:
for name, df in datasets.items():
    print(f"\n📁 {name} — Shape: {df.shape}")
    print(df.head(3))


📁 Brent — Shape: (384, 6)
         Date    Close     High      Low     Open     Volume
0  2024-01-02  6314.25  6576.33  6290.12  6424.07  2378845.5
1  2024-01-03  6318.37  6580.63  6294.23  6428.27  2380401.0
2  2024-01-04  6323.01  6585.46  6298.85  6432.99  2382147.8

📁 WTI — Shape: (384, 6)
         Date    Close     High      Low     Open       Volume
0  2024-01-02  5855.80  6127.04  5829.17  5966.46  27539394.51
1  2024-01-03  5859.63  6131.05  5832.99  5970.36  27557402.20
2  2024-01-04  5863.93  6135.54  5837.27  5974.74  27577624.55

📁 Nifty50 — Shape: (3079, 8)
         Date   ^NSEI_Open   ^NSEI_High    ^NSEI_Low  ^NSEI_Close  \
0  2013-01-02  5982.600098  6006.049805  5982.000000  5993.250000   
1  2013-01-03  6015.799805  6017.000000  5986.549805  6009.500000   
2  2013-01-04  6011.950195  6020.750000  5981.549805  6016.149902   

   ^NSEI_Adj Close  ^NSEI_Volume   Ticker  
0      5993.250000             0  Nifty50  
1      6009.500000             0  Nifty50  
2      6016.1

In [20]:
# Clean and rename
brent = brent[["Date", "BZ=F_Close"]].rename(columns={"Close": "brent_price"})
nifty50 = nifty50[["Date", "^NSEI_Close"]].rename(columns={"Close": "nifty_price"})

KeyError: "['BZ=F_Close'] not in index"

In [ ]:
# Merge on Date
df = pd.merge(brent, nifty50, on="Date")
df = df.sort_values("Date").dropna()

In [ ]:
df

,Date,BZ=F_Close,^NSEI_Close
0,2013-01-02,112.470001,5993.250000
1,2013-01-03,112.139999,6009.500000
2,2013-01-04,111.309998,6016.149902
3,2013-01-07,111.400002,5988.399902
4,2013-01-08,111.940002,6001.700195
...,...,...,...
2977,2025-07-09,70.190002,25476.099609
2978,2025-07-10,68.639999,25355.250000
2979,2025-07-11,70.360001,25149.849609
2980,2025-07-14,69.209999,25082.300781


In [ ]:
# Calculate returns and volatility
df['brent_ret'] = df['brent_price'].pct_change()
df['nifty_ret'] = df['nifty_price'].pct_change()
df['brent_vol_10d'] = df['brent_ret'].rolling(10).std()
df['nifty_vol_10d'] = df['nifty_ret'].rolling(10).std()
df = df.dropna().reset_index(drop=True)

KeyError: 'brent_price'